In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

## やりたいこと
+ それぞれの人について重複するスケジュールになってしまっているproj_idを求める。(順序も含めて)
    + 端点が一致する場合も含む
    + 包含する場合は除く

In [26]:
%%sql
drop view if exists tbl10;
drop table if exists emp_project;
CREATE TABLE emp_project (
    empno varchar NULL,
    ename varchar NULL,
    proj_id integer NULL,
    proj_start date NULL,
    proj_end date NULL
);

INSERT INTO emp_project (empno,ename,proj_id,proj_start,proj_end) VALUES
     ('7782','CLARK','1','2005-01-16','2005-01-18'),
     ('7782','CLARK','4','2005-01-19','2005-01-24'),
     ('7782','CLARK','7','2005-01-22','2005-01-25'),
     ('7782','CLARK','10','2005-01-25','2005-01-28'),
     ('7782','CLARK','13','2005-01-28','2005-02-02'),
     ('7839','KING','2','2005-01-17','2005-01-21'),
     ('7839','KING','8','2005-01-23','2005-01-25'),
     ('7839','KING','14','2005-01-29','2005-01-30'),
     ('7839','KING','11','2005-01-26','2005-01-27'),
     ('7839','KING','5','2005-01-20','2005-01-24'),
     ('7939','KING','17','2005-01-20','2005-01-30'),
     ('7934','MILLER','3','2005-01-18','2005-01-22'),
     ('7934','MILLER','12','2005-01-27','2005-01-28'),
     ('7934','MILLER','15','2005-01-30','2005-02-03'),
     ('7934','MILLER','9','2005-01-24','2005-01-27'),
     ('7934','MILLER','6','2005-01-21','2005-01-23'),
     ('7934','MILLER','16','2005-01-22','2005-01-23');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
17 rows affected.


[]

In [32]:
%%sql
select e1.empno, e1.ename,
    e1.proj_id || ' < ' || e2.proj_id as overlaps,
    e1.proj_start as e1_start, e1.proj_end as e1_end,
    e2.proj_start as e2_start, e2.proj_end as e2_end
from emp_project as e1, emp_project as e2
where e1.empno = e2.empno and
e1.proj_id <> e2.proj_id and
e1.proj_start < e2.proj_start and
e1.proj_end < e2.proj_end and
(e1.proj_end >= e2.proj_start and e2.proj_end >= e1.proj_start)

*  postgresql://padawan:***@db:5432/dsdojo_db
8 rows affected.


empno,ename,overlaps,e1_start,e1_end,e2_start,e2_end
7782,CLARK,4 < 7,2005-01-19,2005-01-24,2005-01-22,2005-01-25
7782,CLARK,7 < 10,2005-01-22,2005-01-25,2005-01-25,2005-01-28
7782,CLARK,10 < 13,2005-01-25,2005-01-28,2005-01-28,2005-02-02
7839,KING,2 < 5,2005-01-17,2005-01-21,2005-01-20,2005-01-24
7839,KING,5 < 8,2005-01-20,2005-01-24,2005-01-23,2005-01-25
7934,MILLER,3 < 16,2005-01-18,2005-01-22,2005-01-22,2005-01-23
7934,MILLER,3 < 6,2005-01-18,2005-01-22,2005-01-21,2005-01-23
7934,MILLER,9 < 12,2005-01-24,2005-01-27,2005-01-27,2005-01-28


In [41]:
%%sql
-- 3<16は3<6の中に包含されている
select empno, ename,
    proj_id || ' < ' || nxt_proj_id as overlaps
from (
    select *,
        lead(proj_start, 1) over (
            partition by ename
            order by proj_end, proj_start
        ) as nxt_proj_start,
        lead(proj_end, 1) over (
            partition by ename
            order by proj_end, proj_start
        ) as nxt_proj_end,
        lead(proj_id,1) over (
            partition by ename
            order by proj_end, proj_start
        ) as nxt_proj_id
    from emp_project
) as tmp
where nxt_proj_start between proj_start and proj_end and
nxt_proj_end > proj_end

*  postgresql://padawan:***@db:5432/dsdojo_db
7 rows affected.


empno,ename,overlaps
7782,CLARK,4 < 7
7782,CLARK,7 < 10
7782,CLARK,10 < 13
7839,KING,2 < 5
7839,KING,5 < 8
7934,MILLER,3 < 6
7934,MILLER,9 < 12


+ 各人でoverlapする期間を求める